In [6]:
%matplotlib qt

In [7]:
# Imports
import numpy as np
from PIL import Image
import tkinter as tk
from tkinter import simpledialog

from PyQt5.QtWidgets import QApplication
from PyQt5.QtGui import QFont

import matplotlib.pyplot as plt
from matplotlib.widgets import PolygonSelector
from matplotlib.path import Path

plt.rcParams.update({
    'font.size': 18,             # base font size
    'axes.titlesize': 26,        # title font
})

In [8]:
class PolygonMaskCreator:
    def __init__(self, image_path):
        self.image = np.array(Image.open(image_path))
        self.height, self.width = self.image.shape[:2]
        self.combined_mask = np.zeros((self.height, self.width), dtype=np.uint8)
        self.current_polygon = []
        self.cid_motion = None
        self.temp_line = None
        self.background = None
        self.poly_count = 0
        self.calibrated = False
        self.scale_factor = None  # pixels per meter

        self.fig, self.ax = plt.subplots()
        self.ax.format_coord = lambda x, y: f"(x, y) = ({x:.0f}, {y:.0f})"
        img = self.ax.imshow(self.image, interpolation='none')
        img.format_cursor_data = lambda val: ""
        self.ax.set_xlabel("x (pixels)")
        self.ax.set_ylabel("y (pixels)")
        self.ax.set_title("Draw calibration line (2 clicks)\n")

        self.line = None
        self.calibration_points = []
        self.cid_click = self.fig.canvas.mpl_connect("button_press_event", self.on_click)
        self.cid_key = None  # store keypress connection ID

        # Maximize window (Qt backend only)
        manager = plt.get_current_fig_manager()
        manager.window.showMaximized()
        manager.window.raise_()  # Try to raise the window to the front
        manager.window.activateWindow()  # Ensure the window is activated and brought to the front

    def on_mouse_move(self, event):
        if not self.temp_line or not event.inaxes:
            return

        x0, y0 = self.calibration_points[0]
        x1, y1 = event.xdata, event.ydata

        self.temp_line.set_data([x0, x1], [y0, y1])
        self.fig.canvas.restore_region(self.background)
        self.ax.draw_artist(self.temp_line)
        self.fig.canvas.blit(self.ax.bbox)

    def on_click(self, event):
        if event.inaxes != self.ax:
            return

        self.calibration_points.append((event.xdata, event.ydata))
        self.ax.plot(event.xdata, event.ydata, 'o', color='cyan', markersize=5, mec='cyan', mfc='cyan', alpha=0.8)

        if len(self.calibration_points) == 1:
            self.temp_line, = self.ax.plot([], [], color='cyan', linestyle='-', linewidth=2, alpha=0.8)
            self.background = self.fig.canvas.copy_from_bbox(self.ax.bbox)
            self.cid_motion = self.fig.canvas.mpl_connect("motion_notify_event", self.on_mouse_move)

        elif len(self.calibration_points) == 2:
            x0, y0 = self.calibration_points[0]
            x1, y1 = self.calibration_points[1]

            # Final line
            self.ax.plot([x0, x1], [y0, y1], color='cyan', linewidth=2, alpha=0.8)

            distance_px = np.hypot(x1 - x0, y1 - y0)
            root = tk.Tk()
            root.withdraw()
            real_distance = float(simpledialog.askstring("Length input", "Enter real-world distance (in meters):"))
            self.scale_factor = distance_px / real_distance

            self.calibrated = True

            self.fig.canvas.mpl_disconnect(self.cid_click)
            self.fig.canvas.mpl_disconnect(self.cid_motion)

            if self.temp_line:
                self.temp_line.remove()
                self.temp_line = None

            self.fig.canvas.draw_idle()
            self.start_polygon_selector()

    def start_polygon_selector(self):
        # Disconnect previous key press handler if it exists
        if self.cid_key:
            self.fig.canvas.mpl_disconnect(self.cid_key)

        self.cid_key = self.fig.canvas.mpl_connect("key_press_event", self.on_key_press)

        self.ax.set_title("Draw polygon → Press ENTER to confirm\n")
        self.selector = PolygonSelector(
            self.ax, self.on_select, useblit=True,
            props=dict(color='red', linestyle='-', linewidth=2, alpha=0.8),
            handle_props=dict(marker='o', markersize=5, mec='red', mfc='red', alpha=0.8)
        )
        self.fig.canvas.mpl_connect("key_press_event", self.on_key_press)

    def on_select(self, verts):
        self.current_polygon = verts

    def on_key_press(self, event):
        if event.key == 'enter' and self.current_polygon:
            self.add_polygon_to_mask()
            self.poly_count += 1

            poly_m = np.array(self.current_polygon) / self.scale_factor
            x = poly_m[:, 0]
            y = poly_m[:, 1]
            area_km2 = 0.5 * np.abs(np.dot(x, np.roll(y, -1)) - np.dot(y, np.roll(x, -1))) /1e6

            # Compute centroid of the polygon
            poly_array = np.array(self.current_polygon)
            centroid_x = poly_array[:, 0].mean()
            centroid_y = poly_array[:, 1].mean()

            # Plot area text at the centroid
            self.ax.text(
                centroid_x, centroid_y,
                f"{self.poly_count}: {area_km2:.0f} km²",
                color='red', fontsize=10, ha='center', va='center', weight='bold',
                bbox=dict(facecolor='white', edgecolor='red', boxstyle='round,pad=0.2', alpha=0.7)
            )

            # Draw the confirmed polygon permanently
            poly_patch = plt.Polygon(self.current_polygon, closed=True, fill=False, edgecolor='red', linewidth=2)
            self.ax.add_patch(poly_patch)

            # Reset title
            self.ax.set_title("Draw next polygon → Press ENTER\n")

            # Fully remove old selector
            self.selector.disconnect_events()
            del self.selector

            # Create new selector
            self.current_polygon = []
            self.selector = PolygonSelector(
                self.ax, self.on_select, useblit=True,
                props=dict(color='red', linestyle='-', linewidth=2, alpha=0.8),
                handle_props=dict(marker='o', markersize=5, mec='red', mfc='red', alpha=0.8)
            )

            # Reconnect key handler
            if self.cid_key:
                self.fig.canvas.mpl_disconnect(self.cid_key)
            self.cid_key = self.fig.canvas.mpl_connect("key_press_event", self.on_key_press)

            self.fig.canvas.draw_idle()


    def add_polygon_to_mask(self):
        # Create a grid of coordinates
        y_grid, x_grid = np.meshgrid(np.arange(self.height), np.arange(self.width), indexing='ij')
        coords = np.vstack((x_grid.ravel(), y_grid.ravel())).T

        # Don't flip Y — everything is already in image coordinates
        path = Path(self.current_polygon)
        inside = path.contains_points(coords).reshape((self.height, self.width)).astype(np.uint8)

        self.combined_mask = np.logical_or(self.combined_mask, inside).astype(np.uint8)

In [9]:
pmc = PolygonMaskCreator("HornsRev123.png")

In [10]:
fig, ax = plt.subplots(figsize=(12, 12))

# Maximize window (Qt backend only)
manager = plt.get_current_fig_manager()
manager.window.showMaximized()

extent = [0, pmc.width / pmc.scale_factor,  # x: in meters
          pmc.height / pmc.scale_factor, 0]  # y: in meters (top-down view)

ax.imshow(pmc.combined_mask, cmap='gray', extent=extent)

#ax.invert_yaxis()

ax.set_xlabel("Distance (m)")
ax.set_ylabel("Distance (m)")
ax.set_title("\nCombined Mask in Real-World Coordinates\n")
plt.tight_layout()
plt.subplots_adjust(bottom=0.05)
plt.grid(True)
plt.show()